In [1]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import prettyplotlib as ppl

In [2]:
# read in original Kaluz file, add columns we need and fill with nans so they can be overwritten

df_orig = pd.read_csv('final_data_files/kaluzny_2004_rrl_reformat.csv',index_col='id')
phot_bands = ['j','h','k','3','4']
colnames = [['mag_{}'.format(i), 'merr_{}'.format(i)] for i in phot_bands]
colnames += [['resid_{}'.format(i)] for i in phot_bands]
colnames += [['photfeh','photfeh_err']]
colnames = sum(colnames, [])
for col in colnames:
    df_orig[col] = np.nan

In [3]:
# ingest the oddly formatted JHK data

# Light curve file columns:
# MHJD is modified heliocentric julian date  =  HJD - 2400000.5
# MAG is the mag on the 2MASS system
# MER is the mag err
# SRC is the src of photometry, 2 = Kaluzny Vband   3 = Kaluzny Bband
# -1 = Fourstar H    -2 = Fourstar Ks  -5 = Fourstar J
# NUM is the number of averaged measurements that went into MAG.  Should
# be unity for everything

jhk_dir = '../Omega_Cen_phot/'
ids = df_orig.index.astype(int).astype(str).str.zfill(3).values
for obj in ids:
    fname = os.path.join(jhk_dir, '{}.txt'.format(obj))
    if os.path.exists(fname):
        row = df_orig[df_orig.index == float(obj)]
        per = row.per.values[0]
        df = pd.read_csv(fname, delim_whitespace=True, names=['mjd','mag','merr','band','n'])
        df_j = df[df.band==-5]
        df_h = df[df.band==-1]
        df_k = df[df.band==-2]
        if df_j.shape[0] > 0:
            mag_j = np.log10(np.mean(10**df_j.mag))
            merr_j = np.sqrt(np.sum(df_j.merr**2))/df_j.shape[0]
            df_j_new = pd.DataFrame({'mag_j':mag_j,'merr_j':merr_j},index=[float(obj)])
            df_orig.update(df_j_new)
        if df_h.shape[0] > 0:
            mag_h = np.log10(np.mean(10**df_h.mag))
            merr_h = np.sqrt(np.sum(df_h.merr**2))/df_h.shape[0]
            df_h_new = pd.DataFrame({'mag_h':mag_h,'merr_h':merr_h},index=[float(obj)])
            df_orig.update(df_h_new)
        if df_k.shape[0] > 0:
            mag_k = np.log10(np.mean(10**df_k.mag))
            merr_k = np.sqrt(np.sum(df_k.merr**2))/df_k.shape[0]
            df_k_new = pd.DataFrame({'mag_k':mag_k,'merr_k':merr_k},index=[float(obj)])
            df_orig.update(df_k_new)

In [4]:
# ingest gloess mags for 3.6 and 4.5

cal_dir = '../calibrated_files/'
for filt in ['3p6','4p5']:
    cal_files = glob.glob(cal_dir + '*{}um_rrl_matched_to_kaluzny.cut'.format(filt))
    for f in cal_files:
        mag = 'mag_{}'.format(filt[0])
        merr = 'merr_{}'.format(filt[0])
        df = pd.read_csv(f,index_col='ID')
        #print df.als_id
        df.index = df.index.str.split('V').str.get(-1).astype(float)
        df[mag] = df.Gloess_mag
        df[merr] = df.Gloess_err
        df_orig.update(df[[mag,merr]])

In [7]:
# ingest metallicities (rey only at this point)

rey_csv = pd.read_csv('final_data_files/rey_2000.cat',delim_whitespace=True,index_col='id')
df_orig.update(rey_csv[['photfeh','photfeh_err']])

In [10]:
df_orig.to_csv('final_data_files/all_possible_photometry.csv')